In [1]:
import persistence_homology as ph

In [2]:
import csv
import os

In [3]:
def read_csv(path):
    with open(path, newline="") as f:
        reader = csv.reader(f)
        output = []
        for row in reader:
            values = []
            for value in row:
                if len(row) == 3:
                    values.append(float(value))
                elif len(row) == 2:
                    values.append(int(value))
            output.append(values)
        return output
def get_subfolders(path):
    """
    Return a list of names of all subdirectories in the given path.
    """
    return [
        name for name in os.listdir(path)
        if os.path.isdir(os.path.join(path, name))
    ]

In [4]:
segmentation_folder_path = "./lung_segmentations"
lung_segmentations = get_subfolders(segmentation_folder_path)

# Filtration Algorithm Start

In [5]:
for lung_segmentation in lung_segmentations:
    folder_path = './lung_segmentations/{}'.format(lung_segmentation)
    edges_path = '/edges.csv'
    vertices_path = '/vertices.csv'
    vertices = read_csv(folder_path + vertices_path)
    edges = read_csv(folder_path + edges_path)
    directions = ph.generate_sphere_points(20, 5, 0.0000001)
    print("Lung Segmentation-{}:".format(lung_segmentation))
    for direction in directions:
        bz = ph.BettiZero(direction, vertices, edges)
        components, mergers, verts, births = bz.compute_persistence()
        intervals = ph.compute_intervals(births, mergers)
        print("Direction: {}. Largest Bar: {}.".format(direction, ph.compute_largest_bar(intervals)))

Lung Segmentation-Lung segmentation-AI-25:
Direction: [-1.  0.  0.]. Largest Bar: (31.0, [-325.5, -294.5]).
Direction: [-0.95105652 -0.30901699  0.        ]. Largest Bar: (24.871502622695857, [-397.6627449841028, -372.79124236140694]).
Direction: [-0.95105652  0.30901699  0.        ]. Largest Bar: (25.465531078275433, [-212.53755982033897, -187.07202874206354]).
Direction: [-0.80901699 -0.58778525  0.        ]. Largest Bar: (27.379716488251006, [-450.0924237703595, -422.7127072821085]).
Direction: [-0.80901699  0.          0.58778525]. Largest Bar: (30.293684122300704, [-180.38140112185175, -150.08771699955105]).
Direction: [-0.80901699  0.         -0.58778525]. Largest Bar: (43.69341587602952, [-362.3041857962633, -318.6107699202338]).
Direction: [-0.80901699  0.58778525  0.        ]. Largest Bar: (26.22910461107884, [-70.4135735049002, -44.18446889382136]).


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x103853fa0>>
Traceback (most recent call last):
  File "/Users/kiyoshitakeuchi/miniforge3/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 